<a href="https://colab.research.google.com/github/nowickimichal/gestures_and_author_classification/blob/main/data_spliter_gesture_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import xml.dom.minidom
import matplotlib.pyplot as plt
from google.colab import drive
from scipy import signal
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
path='/content/gdrive/MyDrive/xml/xml_logs/'
people=['s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11']
pace=['fast','medium','slow']

n=100
type_of_data= 'gestures'
data = np.zeros([2*n,1152])
list_names=[]
list_people=[]
list_time=[]
list_speed=[]
data=[]

m=0

for subpath in people:
  path2=path+subpath+'/'
 
  for speedi in pace:
    path3=path2+speedi
    files = os.listdir(path3)
    print(path3)
    for i in range (0,len(files)):
      
      fname=files[i]
      doc = xml.dom.minidom.parse(path3+'/'+fname);
      point_val = doc.getElementsByTagName("Point")
      gesture=doc.getElementsByTagName('Gesture')
      T_vector = np.zeros((point_val.length,1),int)
      Y_vector = np.zeros((point_val.length,1),int)
      X_vector = np.zeros((point_val.length,1),int)
      for skill in gesture:
        name=skill.getAttribute("Name")
        name=name[0:len(name)-2]
        person=skill.getAttribute("Subject")
        time=skill.getAttribute("Millseconds")
        speed=skill.getAttribute("Speed")
      
      j=0
      for skill in point_val:
        T_vector[j]=skill.getAttribute("T")
        Y_vector[j]=skill.getAttribute("Y")
        X_vector[j]=skill.getAttribute("X")
        j=j+1
      #preprocessing
      #długość sygnału przepróbkoanego n=100
      X_resampled=signal.resample_poly(X_vector, n, len(Y_vector), padtype='line')
      Y_resampled=signal.resample_poly(Y_vector, n, len(Y_vector), padtype='line')
      #standaryzacja
      scaler = StandardScaler()
      scaler.fit(X_resampled)
      scaler.mean_
      scaler.transform(X_resampled)
      standard_X=scaler.transform(X_resampled)

      scaler.fit(Y_resampled)
      scaler.mean_
      scaler.transform(Y_resampled)
      standard_Y=scaler.transform(Y_resampled)
      #filtracja
      b, a = signal.butter(3, 0.4)
      X_fil = signal.filtfilt(b, a, standard_X.T)
      X_fil=X_fil[0,:]
      Y_fil = signal.filtfilt(b, a, standard_Y.T)
      Y_fil=Y_fil[0,:]
      
      #agregacja
      data_scaled = np.concatenate((X_fil,Y_fil))
      list_names.append(name)     
      data.append(data_scaled)
      m=m+1


/content/gdrive/MyDrive/xml/xml_logs/s01/fast
/content/gdrive/MyDrive/xml/xml_logs/s01/medium
/content/gdrive/MyDrive/xml/xml_logs/s01/slow
/content/gdrive/MyDrive/xml/xml_logs/s02/fast
/content/gdrive/MyDrive/xml/xml_logs/s02/medium
/content/gdrive/MyDrive/xml/xml_logs/s02/slow
/content/gdrive/MyDrive/xml/xml_logs/s03/fast
/content/gdrive/MyDrive/xml/xml_logs/s03/medium
/content/gdrive/MyDrive/xml/xml_logs/s03/slow
/content/gdrive/MyDrive/xml/xml_logs/s04/fast
/content/gdrive/MyDrive/xml/xml_logs/s04/medium
/content/gdrive/MyDrive/xml/xml_logs/s04/slow
/content/gdrive/MyDrive/xml/xml_logs/s05/fast
/content/gdrive/MyDrive/xml/xml_logs/s05/medium
/content/gdrive/MyDrive/xml/xml_logs/s05/slow
/content/gdrive/MyDrive/xml/xml_logs/s06/fast
/content/gdrive/MyDrive/xml/xml_logs/s06/medium
/content/gdrive/MyDrive/xml/xml_logs/s06/slow
/content/gdrive/MyDrive/xml/xml_logs/s07/fast
/content/gdrive/MyDrive/xml/xml_logs/s07/medium
/content/gdrive/MyDrive/xml/xml_logs/s07/slow
/content/gdrive/MyDr

In [5]:
df_data=pd.DataFrame(data)
df_names=pd.DataFrame(list_names)
df_data=df_names.join(df_data,lsuffix='Y', rsuffix='X')

In [6]:
df_data.to_csv('/content/gdrive/MyDrive/master_thesis_data/gesture_data.csv')